In [46]:
import pandas as pd

df = pd.read_csv('tahkeer_data_cleaned.csv')

In [47]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier

class BaggingClassifier:
    def __init__(self, n_estimators=100, max_depth=None):
        self.n_estimators = n_estimators
        self.max_depth = max_depth  
        self.models = [DecisionTreeClassifier(max_depth=self.max_depth) for _ in range(n_estimators)] 
        
        
    def fit(self, x, y):
        for model in self.models:
            indices = np.random.choice(len(x), len(x), replace=True)
            x_subset, y_subset = x.iloc[indices], y.iloc[indices] 
            model.fit(x_subset, y_subset)
        
        return self
            

    def predict(self, x, threshold=0.5):
        pred = np.zeros((len(x), self.n_estimators))
        for i, model in enumerate(self.models):
            pred[:, i] = model.predict(x)
        avg_predictions = np.mean(pred, axis=1)
        binary_predictions = (avg_predictions >= threshold).astype(int)
        return binary_predictions
    
    def get_params(self, deep=True):
        return {
            'n_estimators': self.n_estimators,
            'max_depth': self.max_depth,
        }
        
    def set_params(self, **params):
        if not params:
            return self

        for param, value in params.items():
            setattr(self, param, value)
        
        return self

In [48]:
columns = df.columns.tolist()
columns.remove("smoking")
features_x = df[columns]
class_y = df["smoking"]

In [49]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

xtrain, xtest, ytrain, ytest = train_test_split(features_x, class_y, test_size=0.30, shuffle=False, train_size=0.70)

In [51]:
class AdaBoost:
    def __init__(self, n_estimators=50, max_depth=4):
        self.n_estimators = n_estimators
        self.alphas = []
        self.models = []
        self.max_depth = max_depth

    def fit(self, x, y):
        x = np.array(x)
        y = np.array(y)
        n_samples, n_features = x.shape
        weights = np.ones(n_samples) / n_samples

        for _ in range(self.n_estimators):
            model = DecisionTreeClassifier(max_depth=self.max_depth)
            model.fit(x, y, sample_weight=weights)
            y_pred = model.predict(x)
            weighted_error = np.sum(weights[y_pred != y])
            if weighted_error >= 0.5:
                break

            alpha = 0.5 * np.log((1.0 - weighted_error) / max(weighted_error, 1e-10))
            self.alphas.append(alpha)
            self.models.append(model)

            # Update weights
            weights *= np.exp(-alpha * y * y_pred)
            weights /= np.sum(weights)
        
        return self

    def predict(self, x):
        x = np.array(x)
        pred = np.zeros(len(x))
        for alpha, model in zip(self.alphas, self.models):
            pred += alpha * model.predict(x)
        return np.sign(pred)
    
    def get_params(self, deep=True):
        return {
            'n_estimators': self.n_estimators,
            'max_depth': self.max_depth,
        }
        
    def set_params(self, **params):
        if not params:
            return self

        for param, value in params.items():
            setattr(self, param, value)
        
        return self

In [63]:
class RandomForestClassifier:
    def __init__(self, n_estimators=100, max_depth=None, min_samples_split=2, min_samples_leaf=1):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.models = []

    def fit(self, x, y):
        x = np.array(x)
        y = np.array(y)
        n_samples, n_features = x.shape
        for _ in range(self.n_estimators):
            # Randomly select a subset of features
            selected_features = np.random.choice(n_features, size=int(np.sqrt(n_features)), replace=False)
            x_subset = x[:, selected_features]

            # Create a decision tree with random features
            tree = DecisionTreeClassifier(
                max_depth=self.max_depth,
                min_samples_split=self.min_samples_split,
                min_samples_leaf=self.min_samples_leaf
            )
            tree.fit(x_subset, y)
            self.models.append((tree, selected_features))
        return self

    def predict(self, x):
        x = np.array(x)
        pred = np.zeros((x.shape[0], self.n_estimators))
        for i, (tree, selected_features) in enumerate(self.models):
            x_subset = x[:, selected_features]
            pred[:, i] = tree.predict(x_subset)

        # Use majority voting for the final prediction
        final_predictions = np.apply_along_axis(lambda x: np.bincount(x.astype(int)).argmax(), axis=1, arr=pred)
        return final_predictions
    
    def get_params(self, deep=True):
        return {
            'n_estimators': self.n_estimators,
            'max_depth': self.max_depth,
            'min_samples_split': self.min_samples_split,
            'min_samples_leaf': self.min_samples_leaf
        }
        
    def set_params(self, **params):
        if not params:
            return self

        for param, value in params.items():
            setattr(self, param, value)
        
        return self

In [50]:
bagging_model = BaggingClassifier(n_estimators=100, max_depth=60)
bagging_model.fit(xtrain, ytrain)

bagging_predictions = bagging_model.predict(xtest)
bagging_accuracy = accuracy_score(ytest, bagging_predictions)
print(f"Bagging Accuracy: {bagging_accuracy}")

Bagging Accuracy: 0.7312059431851758


In [52]:
adaboost = AdaBoost(n_estimators=300)
adaboost.fit(xtrain, ytrain)

predictions = adaboost.predict(xtest)

accuracy = accuracy_score(ytest, predictions)
print(f'Boosting Accuracy: {accuracy}')

Accuracy: 0.7195770545776877


In [61]:
random_forest = RandomForestClassifier(n_estimators=150, max_depth=3, min_samples_split=2, min_samples_leaf=1)
random_forest.fit(xtrain, ytrain)

predictions = random_forest.predict(xtest)
accuracy = accuracy_score(ytest, predictions)
print(f'Random Forest Accuracy: {accuracy}')

Random Forest Accuracy: 0.7144696298172302


# Hyperparameter Tuning
We will use grid search and randomized search methods to choose better hyperparameters

## Tuning Random Forest model

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

params = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 5, 10, 20],
    'min_samples_split': [2, 3, 4, 6, 8],
    'min_samples_leaf': [1, 2, 3, 4],
}

rf_model = RandomForestClassifier()
grid_search = GridSearchCV(estimator=rf_model, param_grid=params, cv=5, scoring='accuracy')
grid_search.fit(xtrain, ytrain)

print("Grid search: ")
best_params = grid_search.best_params_
best_score = grid_search.best_score_

best_model = grid_search.best_estimator_
test_score = best_model.score(xtest, ytest)

print("Best Parameters:", best_params)
print("Best Score:", best_score)
print("Test Set Score:", test_score)

In [ ]:
random_search = RandomizedSearchCV(estimator=rf_model, param_distributions=params, n_iter=10, cv=5, scoring='accuracy', random_state=42)
random_search.fit(xtrain, ytrain)

print("Random Search: ")
best_params = random_search.best_params_
best_score = random_search.best_score_

best_model = random_search.best_estimator_
test_score = best_model.score(xtest, ytest)

print("Best Parameters:", best_params)
print("Best Score:", best_score)
print("Test Set Score:", test_score)